In [1]:
library(Seurat)
library(SeuratDisk)
library(ggplot2)
library(patchwork)
library(dplyr)
library(Matrix)

Attaching SeuratObject

Registered S3 method overwritten by 'cli':
  method     from         
  print.boxx spatstat.geom

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
reference = LoadH5Seurat("../Data/pbmc_multimodal.h5seurat")

Validating h5Seurat file

Initializing ADT with data

Adding counts for ADT

Initializing SCT with data

Adding counts for SCT

Adding variable feature information for SCT

Adding miscellaneous information for SCT

Adding reduction apca

Adding cell embeddings for apca

Adding feature loadings for apca

Adding miscellaneous information for apca

Adding reduction aumap

Adding cell embeddings for aumap

Adding miscellaneous information for aumap

Adding reduction pca

Adding cell embeddings for pca

Adding feature loadings for pca

Adding miscellaneous information for pca

Adding reduction spca

Adding cell embeddings for spca

Adding feature loadings for spca

Adding miscellaneous information for spca

Adding reduction umap

Adding cell embeddings for umap

Adding miscellaneous information for umap

Adding reduction wnn.umap

Adding cell embeddings for wnn.umap

Adding miscellaneous information for wnn.umap

Adding graph wknn

Adding graph wsnn

Adding command information

Adding cell-

In [3]:
gene_matrix = readMM("../Data/H1N1/gene_data.mtx")
meta_data = read.table("../Data/H1N1/meta_data.txt", sep = ',', row.names = 1, header = T)

colnames(gene_matrix) = rownames(meta_data)
rownames(gene_matrix) = read.table("../Data/H1N1/gene_names.txt", sep = ',')$V2[-1]

In [4]:
runseurat4 = function(indices_train, indices_test, reference, gene_test, test_meta) {
    start = proc.time()[3]
    
    gene_matrix = reference@assays$SCT@counts[,indices_train]
    protein_matrix = reference@assays$ADT@counts[,indices_train]

    cell_indices = colSums(gene_matrix >= 1) >= 200

    gene_matrix = gene_matrix[, cell_indices]
    protein_matrix = protein_matrix[, cell_indices]

    print(dim(gene_matrix))

    meta_data = reference@meta.data[indices_train,][cell_indices,]

    train_assay = CreateSeuratObject(
    gene_matrix,
    assay = "RNA",
    meta.data = meta_data)

    protein = CreateSeuratObject(
      protein_matrix,
      assay = "ADT",
      meta.data = meta_data)

    keep_features = rownames(train_assay@assays$RNA@counts)[rowSums(train_assay@assays$RNA@counts >= 1) >= 30]
    train_assay = subset(train_assay, features = keep_features)

    keep_features = rownames(protein@assays$ADT@counts)[rowSums(protein@assays$ADT@counts) >= 1]
    protein = subset(protein, features = keep_features)

    train_assay@assays$ADT = protein@assays$ADT #comment this
    
    train_list <- SplitObject(train_assay, split.by = "donor")

    rm(keep_features)
    rm(meta_data)
    rm(cell_indices)
    rm(gene_matrix)
    rm(protein_matrix)
    rm(reference)
    rm(train_assay)
    rm(protein)
    
    for (i in 1:length(train_list)) {
    DefaultAssay(train_list[[i]]) <- 'RNA'
    train_list[[i]] <- suppressWarnings(SCTransform(train_list[[i]], verbose = FALSE))

    print("ADT")
    DefaultAssay(train_list[[i]]) <- 'ADT'
    VariableFeatures(train_list[[i]]) <- rownames(train_list[[i]][["ADT"]])
    train_list[[i]] <- NormalizeData(train_list[[i]], normalization.method = 'CLR', margin = 2, verbose = FALSE) %>% 
         ScaleData()

    }
    
    for (i in 1:length(train_list)) {
    DefaultAssay(train_list[[i]]) <- 'SCT'
    }
    
    train_int_features <- SelectIntegrationFeatures(
        object.list = train_list, nfeatures = 3000)
    
    options(future.globals.maxSize= 30*1024^3)

    train_list <- PrepSCTIntegration(
        object.list = train_list, anchor.features = train_int_features, 
        verbose = FALSE)
    
    train_list <- lapply(X = train_list, FUN = function(x) {
    x <- RunPCA(x, features = train_int_features, verbose = FALSE)
    })
    
    train_anchors <- FindIntegrationAnchors(
    object.list = train_list, reduction = "rpca",
    anchor.features = train_int_features,
    scale=FALSE, normalization.method = 'SCT',
    dims = 1:30)
    
    train_rna_integrated <- IntegrateData(anchorset = train_anchors, dims = 1:30)
    SaveH5Seurat(train_rna_integrated, "train_integrated.h5seurat", overwrite = TRUE)
    rm(train_rna_integrated)
    
    for (i in 1:length(train_list)) {
    DefaultAssay(train_list[[i]]) <- 'ADT'
    }
    
    train_list <- lapply(X = train_list, FUN = function(x) {
    x <- RunPCA(x, reduction.name = 'apca', verbose = FALSE)
    })
    
    for (i in 1:length(train_list)) {
    train_list[[i]]@reductions$rpca <- train_list[[i]]@reductions$pca
    train_list[[i]]@reductions$pca <- train_list[[i]]@reductions$apca
    }
    
    prt_int_features <- rownames(train_list[[1]]@assays$ADT)

    prt_anchors <- FindIntegrationAnchors(
        object.list = train_list, reduction = "rpca",
        anchor.features = prt_int_features,
        scale=FALSE, normalization.method = 'LogNormalize',
        dims = 1:30)

    rm(train_int_features)
    rm(prt_int_features)
    rm(train_list)
    
    train_protein_integrated <- IntegrateData(anchorset = prt_anchors, dims = 1:30)
    train_rna_integrated = LoadH5Seurat("train_integrated.h5seurat")
    
    train_assay <- CreateSeuratObject(
    counts = train_rna_integrated@assays$integrated,
    assay = 'RNA',
    meta.data = train_rna_integrated@meta.data,
    min.cells=0,
    min.features=0)
    
    train_assay@assays$ADT <- train_protein_integrated@assays$integrated
    
    DefaultAssay(train_assay) = "RNA"
    train_assay = ScaleData(train_assay) %>% FindVariableFeatures() %>% RunPCA()

    DefaultAssay(train_assay) <- 'ADT'

    train_assay <- ScaleData(train_assay) %>% FindVariableFeatures() %>% RunPCA(reduction.name = 'apca')

    train_assay <- FindMultiModalNeighbors(
      train_assay, reduction.list = list("pca", "apca"), 
      dims.list = list(1:30, 1:18), modality.weight.name = "RNA.weight"
    )
    
    train_assay <- RunUMAP(train_assay, nn.name = "weighted.nn", return.model=T,
                       reduction.name = "wnn.umap", reduction.key = "wnnUMAP_")

    train_assay <- FindClusters(train_assay, graph.name = "wsnn",
                                algorithm = 3, resolution = 0.2, verbose = FALSE)

    options(repr.plot.width=7, repr.plot.height=7)

    p1 <- DimPlot(train_assay, reduction = 'wnn.umap', group.by = "orig.ident")
    p1
    
    # Run supervised PCA on weighted NN graph
    DefaultAssay(train_assay) <- 'RNA'

    train_assay <- ScaleData(train_assay, assay = 'RNA')
    train_assay <- RunSPCA(train_assay, assay='RNA', graph='wsnn')

    train_assay <- FindNeighbors(
      object = train_assay,
      reduction = "spca",
      dims = 1:50,
      graph.name = "spca.annoy.neighbors", 
      k.param = 50,
      cache.index = TRUE,
      return.neighbor = TRUE,
      l2.norm = TRUE
    )
    
    gene_matrix = gene_test[,indices_test]
    meta_data = test_meta[indices_test,]

    colnames(gene_matrix) = rownames(meta_data)
        
    cell_indices = colSums(gene_matrix >= 1) >= 200

    gene_matrix = gene_matrix[, cell_indices]

    meta_data = meta_data[cell_indices,]
    
    test_assay = CreateSeuratObject(
    gene_matrix,
    assay = "RNA",
    meta.data = meta_data)

    rm(gene_matrix)

    keep_features = rownames(test_assay@assays$RNA@counts)[rowSums(test_assay@assays$RNA@counts >= 1) >= 30]
    test_assay = subset(test_assay, features = keep_features)
    
    test_list <- SplitObject(test_assay, split.by = "sample")
    
    for (i in 1:length(test_list)) {
        DefaultAssay(test_list[[i]]) <- 'RNA'
        test_list[[i]] <- suppressWarnings(SCTransform(test_list[[i]], verbose = FALSE))
    }
    
    for (i in 1:length(test_list)) {
        DefaultAssay(test_list[[i]]) <- 'SCT'
    }

    test_int_features <- SelectIntegrationFeatures(
        object.list = test_list, nfeatures = 3000)

    test_list <- PrepSCTIntegration(
        object.list = test_list, anchor.features = test_int_features, 
        verbose = FALSE)

    test_list <- lapply(X = test_list, FUN = function(x) {
        x <- RunPCA(x, features = test_int_features, verbose = FALSE)
    })
    
    
    test_anchors <- FindIntegrationAnchors(
        object.list = test_list, reduction = "rpca",
        anchor.features = test_int_features,
        scale=FALSE, normalization.method = 'SCT',
        dims = 1:30)

    rm(test_list)
    
    test_rna_integrated <- IntegrateData(anchorset = test_anchors, dims = 1:30)
    
    test_assay <- CreateSeuratObject(
        counts = test_rna_integrated@assays$integrated,
        assay = 'RNA',
        meta.data = test_rna_integrated@meta.data,
        min.cells=0,
        min.features=0)

    rm(test_rna_integrated)

    anchors <- FindTransferAnchors(
      reference = train_assay,
      query = test_assay,
      reference.reduction = "spca",
      dims = 1:50
    )

    test_assay <- TransferData(
      anchorset = anchors, 
      reference = train_assay,
      query = test_assay,
      refdata = list(
        cluster = "wsnn_res.0.2",
        predictedADT = "ADT",
        k.weight = 10)
    )

    test_assay <- IntegrateEmbeddings(
      anchorset = anchors,
      reference = train_assay,
      query = test_assay, 
      new.reduction.name = "ref.spca"
    )

    test_assay <- ProjectUMAP(
      query = test_assay, 
      query.reduction = "ref.spca", 
      reference = train_assay, 
      reference.reduction = "spca", 
      reduction.model = "wnn.umap"
    )
    
    return(proc.time()[3] - start)
}

In [5]:
files = list.files('runtime/train')
# files = files[-4]

train_path = "runtime/train"
test_path = "runtime/test"

times = rep(0, length(files))
names(times) = sapply(files, function(x) as.numeric(substr(x, 4, 6)))
i = 1

for(file in files) {
    train_idx = read.csv(paste0(train_path, '/', file), sep =',')$idx
    test_idx = read.csv(paste0(test_path, '/', file), sep =',')$idx
    
    print(file)
    
    times[i] = runseurat4(train_idx, test_idx, reference, gene_matrix, meta_data)
    i = i + 1
}

[1] "idx0.1.csv"
[1] 20729 16174
[1] "ADT"


Centering and scaling data matrix



[1] "ADT"


Centering and scaling data matrix



[1] "ADT"


...

Finding anchors

	Found 994 anchors

Finding neighborhoods

Finding anchors

	Found 873 anchors

Finding neighborhoods

Finding anchors

	Found 834 anchors

Finding neighborhoods

Finding anchors

	Found 810 anchors

Finding neighborhoods

Finding anchors

	Found 805 anchors

Finding neighborhoods

Finding anchors

	Found 857 anchors

Finding neighborhoods

Finding anchors

	Found 924 anchors

Finding neighborhoods

Finding anchors

	Found 622 anchors

Finding neighborhoods

Finding anchors

	Found 906 anchors

Finding neighborhoods

Finding anchors

	Found 951 anchors

Finding neighborhoods

Finding anchors

	Found 895 anchors

Finding neighborhoods

Finding anchors

	Found 1132 anchors

Finding neighborhoods

Finding anchors

	Found 720 anchors

Finding neighborhoods

Finding anchors

	Found 808 anchors

Finding neighborhoods

Finding anchors

	Found 919 anchors

Finding neighborhoods

Finding anchors

	Found 853 anchors

Finding neighborhoods

Finding anchors

	Found 892 anchors

Fi

Finding neighborhoods

Finding anchors

	Found 6849 anchors

Finding neighborhoods

Finding anchors

	Found 8661 anchors

Finding neighborhoods

Finding anchors

	Found 6939 anchors

Finding neighborhoods

Finding anchors

	Found 7590 anchors

Finding neighborhoods

Finding anchors

	Found 8746 anchors

Finding neighborhoods

Finding anchors

	Found 7786 anchors

Finding neighborhoods

Finding anchors

	Found 7467 anchors

Finding neighborhoods

Finding anchors

	Found 8344 anchors

Finding neighborhoods

Finding anchors

	Found 7724 anchors

Finding neighborhoods

Finding anchors

	Found 9453 anchors

Merging dataset 2 into 8

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Merging dataset 7 into 8 2

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data

Merging dataset 1 into 3

Extracting anchors for merged samples

Finding integration vectors

UMAP will return its model

13:37:49 UMAP embedding parameters a = 0.9922 b = 1.112

13:37:50 Commencing smooth kNN distance calibration using 1 thread

13:37:56 Initializing from normalized Laplacian + noise

13:38:26 Commencing optimization for 200 epochs, with 5615976 positive edges

13:43:23 Optimization finished

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Centering and scaling data matrix

Computing sPCA transformation

Computing nearest neighbors

Only one graph name supplied, storing nearest-neighbor graph only

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Computing within dataset neighborhoods

Finding all pairwise anchors

Finding neighborhoods

Finding anchors

	Found 1650 anchors

Finding neighborhoods

Finding anchors

	Found 1228 anchors

Finding neighborhoods

Finding anchors

	Found 1467 anchors

Finding neighborhoods

Finding anchors

	Found 1086 anchors

Finding 

	Found 966 anchors

Finding neighborhoods

Finding anchors

	Found 1022 anchors

Finding neighborhoods

Finding anchors

	Found 722 anchors

Finding neighborhoods

Finding anchors

	Found 990 anchors

Finding neighborhoods

Finding anchors

	Found 989 anchors

Finding neighborhoods

Finding anchors

	Found 965 anchors

Finding neighborhoods

Finding anchors

	Found 932 anchors

Finding neighborhoods

Finding anchors

	Found 981 anchors

Finding neighborhoods

Finding anchors

	Found 1026 anchors

Finding neighborhoods

Finding anchors

	Found 993 anchors

Finding neighborhoods

Finding anchors

	Found 947 anchors

Finding neighborhoods

Finding anchors

	Found 1152 anchors

Finding neighborhoods

Finding anchors

	Found 888 anchors

Finding neighborhoods

Finding anchors

	Found 965 anchors

Finding neighborhoods

Finding anchors

	Found 1078 anchors

Finding neighborhoods

Finding anchors

	Found 1526 anchors

Finding neighborhoods

Finding anchors

	Found 1301 anchors

Finding neighb

15:05:47 Commencing smooth kNN distance calibration using 1 thread

15:05:48 Initializing by weighted average of neighbor coordinates using 1 thread

15:05:48 Commencing optimization for 67 epochs, with 1063980 positive edges

15:06:12 Finished

Warning message:
“No assay specified, setting assay as RNA by default.”


In [13]:
fig = data.frame(times)
colnames(fig) = 'Seurat 4'
fig

,Seurat 4
,<dbl>
0.1,1599.921
0.2,3198.419
0.4,6419.462
0.5,7559.770
0.8,11501.132
1,15279.783


In [16]:
write.csv(fig, 'runtime/seurat4.csv')